In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Getting FIFA Countries Codes, wiki scrapper

In [32]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_FIFA_country_codes"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)


200


In [33]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
# fifa_codes=soup.find('table',{'class':"wikitable"})
df1 = soup.find_all('table')[0]
df2 = soup.find_all('table')[1]
df3 = soup.find_all('table')[2]
df4 = soup.find_all('table')[3]

In [34]:
codes1=pd.read_html(str(df1))
# convert list to dataframe
codes1=pd.DataFrame(codes1[0])

codes2=pd.read_html(str(df2))
# convert list to dataframe
codes2=pd.DataFrame(codes2[0])

codes3=pd.read_html(str(df3))
# convert list to dataframe
codes3=pd.DataFrame(codes3[0])

codes4=pd.read_html(str(df4))
# convert list to dataframe
codes4=pd.DataFrame(codes4[0])
# print(codes1.head())

In [35]:
frames = [codes1, codes2, codes3, codes4]
country_codes = pd.concat(frames)
country_codes.rename(columns={'Country': 'team', 'Code': 'code'}, inplace=True)
country_codes['country'] = country_codes.loc[:, 'team']
country_codes

,team,code,country
0,Afghanistan,AFG,Afghanistan
1,Albania,ALB,Albania
2,Algeria,ALG,Algeria
3,American Samoa,ASA,American Samoa
4,Andorra,AND,Andorra
...,...,...,...
47,Vietnam,VIE,Vietnam
48,Wales,WAL,Wales
49,Yemen,YEM,Yemen
50,Zambia,ZAM,Zambia


## Getting Sovereign States

In [51]:
# get the response in the form of html
wikiurl2="https://en.wikipedia.org/wiki/List_of_sovereign_states_by_date_of_formation"
table_class2="wikitable sortable tpl-blanktable jquery-tablesorter"
response2=requests.get(wikiurl2)
print(response2.status_code)

200


In [52]:
# parse data from the html into a beautifulsoup object
soup_states = BeautifulSoup(response2.text, 'html.parser')

In [53]:

len(soup_states.find_all('table'))

9

In [54]:
# It is the last table on page
states = soup_states.find_all('table')[7]

In [55]:
sovereign_states=pd.read_html(str(states))
# convert list to dataframe
sovereign_states=pd.DataFrame(sovereign_states[0])

In [56]:
sovereign_states.head()

,Country,Continent,First acquisition of sovereignty,Date of last subordination,Previous governing power,Historical Notes
0,Afghanistan,Asia,1709,1796,Afsharid Empire,2021–present: Islamic Emirate of Afghanistan 2...
1,Albania,Europe,28 Nov 1912,Nov 1944[44],Germany,1992–present: Republic of Albania 1946–92: Peo...
2,Algeria,Africa,202 BC,3 July 1962[46],France,1962–present: People's Democratic Republic of ...
3,Andorra,Europe,7 Sep 1278[48][49],Nov 1944[50],France,1278–present: Principality of Andorra (via Par...
4,Angola,Africa,11 Nov 1975[51],11 Nov 1975,Portugal,1992–present: Republic of Angola 1975–1992: Pe...


In [57]:
sovereign_states = sovereign_states[['Country', 'Continent']]
sovereign_states.columns = sovereign_states.columns.str.lower()
sovereign_states

,country,continent
0,Afghanistan,Asia
1,Albania,Europe
2,Algeria,Africa
3,Andorra,Europe
4,Angola,Africa
...,...,...
190,Venezuela,The Americas
191,Vietnam,Asia
192,Yemen,Asia
193,Zambia,Africa


## Connecting Sovereing States to FIFA teams and codes

In [58]:
# Teams in country_codes (fifa teams), but not in sovereign states countries (because they are not really countries)
print(list(set(country_codes['country'].unique()) - set(sovereign_states['country'].unique())))

['Puerto Rico', 'Northern Ireland', 'England', 'British Virgin Islands', 'Anguilla', 'Gibraltar', 'Cook Islands', 'Congo', 'Guam', 'Scotland', 'New Caledonia', 'Chinese Taipei', 'Czech Republic', 'Turks and Caicos Islands', 'Cayman Islands', 'Republic of Ireland', 'DR Congo', 'Bermuda', 'Montserrat', 'U.S. Virgin Islands', 'American Samoa', 'Kosovo', 'Aruba', 'Faroe Islands', 'Curaçao', 'Macau', 'Tahiti', 'Wales', 'Hong Kong']


In [59]:
# Countries that are in sovereign states, but not in fifa teams 
print(list(set(sovereign_states['country'].unique()) - set(country_codes['country'].unique())))

['Nauru', 'Kiribati', 'Palau', 'Congo, Republic of the', 'Tuvalu', 'Congo, Democratic Republic of the', 'Micronesia, Federated States of', 'United Kingdom', 'Monaco', 'Czechia', 'Ireland', 'Marshall Islands', 'Vatican City']


In [60]:
# Change a few names in country_codes in order to match teams and their country
team_to_change = ['Chinese Taipei', 'Congo', 'England', 'Wales', 'Northern Ireland', 'Scotland' ]
changes = ['Taiwan', 'Republic of the Congo', 'United Kingdom', 'United Kingdom', 'United Kingdom', 'United Kingdom']


In [61]:
country_codes['country'] = country_codes['country'].replace(team_to_change,changes)

In [62]:
country_codes

,team,code,country
0,Afghanistan,AFG,Afghanistan
1,Albania,ALB,Albania
2,Algeria,ALG,Algeria
3,American Samoa,ASA,American Samoa
4,Andorra,AND,Andorra
...,...,...,...
47,Vietnam,VIE,Vietnam
48,Wales,WAL,United Kingdom
49,Yemen,YEM,Yemen
50,Zambia,ZAM,Zambia


## Getting FIFA Confederations and appending to each team(country)

In [63]:
rankings = pd.read_csv('./data/fifa_rankings.csv')

In [64]:
rankings.head()

,rank,country_abrv,total_points,previous_points,rank_change,confederation,rank_date,team
0,1,GER,57.0,0.0,0,UEFA,1992-12-31,Germany
1,96,SYR,11.0,0.0,0,AFC,1992-12-31,Syria
2,97,BFA,11.0,0.0,0,CAF,1992-12-31,Burkina Faso
3,99,LVA,10.0,0.0,0,UEFA,1992-12-31,Latvia
4,100,BDI,10.0,0.0,0,CAF,1992-12-31,Burundi


In [65]:
country_codes.head()

,team,code,country
0,Afghanistan,AFG,Afghanistan
1,Albania,ALB,Albania
2,Algeria,ALG,Algeria
3,American Samoa,ASA,American Samoa
4,Andorra,AND,Andorra


In [66]:
fifa_teams = country_codes.merge(rankings[['team','confederation']], how='left', on='team').drop_duplicates(subset='team',keep='first')
fifa_teams

,team,code,country,confederation
0,Afghanistan,AFG,Afghanistan,AFC
219,Albania,ALB,Albania,UEFA
541,Algeria,ALG,Algeria,CAF
863,American Samoa,ASA,American Samoa,OFC
1068,Andorra,AND,Andorra,UEFA
...,...,...,...,...
62290,Vietnam,VIE,Vietnam,AFC
62611,Wales,WAL,United Kingdom,UEFA
62933,Yemen,YEM,Yemen,AFC
63255,Zambia,ZAM,Zambia,CAF


In [78]:
# send to csv
fifa_teams.to_csv('./data/fifa_teams.csv', index=False)